# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from gconfig import gkey
# Configure gmaps
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv('cities.csv')
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [70]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = cities_df["Humidity"].astype(float)

In [71]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(humidity_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# A max temperature lower than 80 degrees but higher than 70.
cities_df_temp = cities_df.loc[(cities_df['Max Temp']>70)&(cities_df['Max Temp']<80),:]
cities_df_temp.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
18,18,bambous virieux,75,MU,1558378758,74,-20.34,57.76,78.80,11.41
25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93


In [16]:
# Wind speed less than 10 mph and 0 Cloudiness.
cities_df_ideal = cities_df_temp.loc[(cities_df_temp['Wind Speed']<10)&(cities_df_temp['Cloudiness']==0),:]
cities_df_ideal.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59


In [17]:
cities_df_ideal = cities_df_ideal.dropna()
cities_df_ideal.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
cities_df_ideal.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


In [29]:
cities_df_ideal

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [57]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "0,0",  # coords
    "rankby": "distance",
    "type": "hotels",
    "key": gkey,
}

hotel = []
country = []
h_lat = []
h_lng = []
city = []
# use iterrows to iterate through pandas dataframe
for index, row in cities_df_ideal.iterrows():

    # get restaurant type from df
    city_name = row['City']

    # add keyword to params dict
    params['keyword'] = city_name
    
    #update location based on city lat and lon
    params['location'] = str(row['Lat'])+','+str(row['Lng'])

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel to city {city_name} is {results[0]['name']}.")
        
        
        hotel.append(results[0]['name'])
        country.append(results[0]['vicinity'].split(',')[1])
        h_lat.append(results[0]['geometry']['location']['lat'])  #'geometry': {'location': {'lat':
        h_lng.append(results[0]['geometry']['location']['lng'])
        city.append(results[0]['vicinity'].split(',')[0])
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 70: ponta do sol.
Missing field/result... skipping.
------------
Retrieving Results for Index 88: cidreira.
Closest hotel to city cidreira is Cidreira.
------------
Retrieving Results for Index 138: morondava.
Closest hotel to city morondava is Morondava.
------------
Retrieving Results for Index 139: vaini.
Missing field/result... skipping.
------------
Retrieving Results for Index 176: nador.
Closest hotel to city nador is Nador.
------------
Retrieving Results for Index 297: mogok.
Closest hotel to city mogok is Mogok.
------------
Retrieving Results for Index 444: birjand.
Closest hotel to city birjand is Birjand.
------------


In [62]:
hotel_dict = {'City': city,
             'Hotel Name': hotel,
             'Country':country,
             'Lat':h_lat,
             'Lng':h_lng}
hotel_df = pd.DataFrame(hotel_dict)
hotel_df.head()

,City,Hotel Name,Country,Lat,Lng
0,Cidreira - RS,Cidreira,Brazil,-30.176939,-50.208914
1,Morondava,Morondava,Madagascar,-20.290419,44.299996
2,Nador,Nador,Morocco,35.168616,-2.927584
3,Mogok,Mogok,Myanmar (Burma),22.924674,96.506310
4,Birjand,Birjand,South Khorasan Province,32.864904,59.226247


In [72]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [73]:
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(255, 0, 0, 0.4)',
    stroke_color='rgba(255,0,0, 0.4)', scale=10,
    info_box_content=hotel_info
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))